In [65]:
import numpy as np
import scipy
import matplotlib.pyplot as plt

import os
import argparse
import logging

import torch
from torch.optim.lr_scheduler import StepLR
import torchtext

import seq2seq
from seq2seq.trainer import SupervisedTrainer
from seq2seq.models import EncoderRNN, DecoderRNN, Seq2seq
from seq2seq.loss import Perplexity
from seq2seq.optim import Optimizer
from seq2seq.dataset import SourceField, TargetField
from seq2seq.evaluator import Predictor
from seq2seq.util.checkpoint import Checkpoint

ModuleNotFoundError: No module named 'seq2seq.trainer'

In [8]:
ls Nucleosom_data/

 Chereji_2018_Occupancy_H3_CC_V64-chrI-1..230218.bedgraph
 generatetrain.m
 Hu_2014_nucleosome_over_avg_V64-chrI-1..230218.bedgraph
 PreNuc.m
 PreNuc.m~
'Reference sequence-chrI-1..230218.fasta'
'Reference sequence-chrI-1..230218.mm'


In [78]:
# Load dataset
f = open('Nucleosom_data/Chereji_2018_Occupancy_H3_CC_V64-chrI-1..230218.bedgraph','r')
f.readline()
hist_occ_list = [[float(n) for n in l[1:].replace('\n','').split('\t') ] for l in f]
f.close()

In [81]:
M = int(hist_occ_list[-1][1])
hist_occ = np.zeros(M)
for h in hist_occ_list:
    hist_occ[int(h[0]):int(h[1])] = h[2]

In [83]:
hist_occ.shape

(230218,)

In [84]:
len(sequence)

230218

In [ ]:
class dataset_h5(Dataset):
    def __init__(self,histone_file,fasta_file):
        super(dataset_h5, self).__init__()
        
        #Loading FASTA file
        f = open(fasta_file,'r')
        f.readline() #Remove header
        self.sequence=''
        for l in f:
            self.sequence  = self.sequence + l.replace('\n','').lower()
        
        f = open(histone_file,'r')
        f.readline() #Remove header
        hist_occ_list = [[float(n) for n in l[1:].replace('\n','').split('\t') ] for l in f]
        f.close()
        
        self.M = int(hist_occ_list[-1][1])
        self.hist_occ = np.zeros(self.M)
        for h in hist_occ_list:
            self.hist_occ[int(h[0]):int(h[1])] = h[2]
            
        print('(Fasta len, Nucleosom len) =  ({0},{1})'.format(len(self.sequence),len(hist_occ)))
    
    def OHE(self,seq):
        ohe = np.zeros((4,len(seq)))
        for nuc in ['a','c','t','g']:
            ohe[seq==nuc] = 1
        
        return ohe

    def __getitem__(self, index):
        
        return self.hist_occ[index],OHE(self.sequence[index])
    
    def __len__(self):
        return self.M
    
fasta_file = 'Nucleosom_data/Reference sequence-chrI-1..230218.fasta'
histone_file = 'Nucleosom_data/Chereji_2018_Occupancy_H3_CC_V64-chrI-1..230218.bedgraph'


In [ ]:
# Prepare loss
loss = Perplexity()
if torch.cuda.is_available():
    loss.cuda()

# Initialize model
hidden_size=128
bidirectional = True

encoder = EncoderRNN(len(src.vocab), max_len, hidden_size,
                     bidirectional=bidirectional, variable_lengths=True)
decoder = DecoderRNN(len(tgt.vocab), max_len, hidden_size * 2 if bidirectional else hidden_size,
                     dropout_p=0.2, use_attention=True, bidirectional=bidirectional,
                     eos_id=tgt.eos_id, sos_id=tgt.sos_id)

seq2seq = Seq2seq(encoder, decoder)
if torch.cuda.is_available():
    seq2seq.cuda()

for param in seq2seq.parameters():
    param.data.uniform_(-0.08, 0.08)

# Optimizer and learning rate scheduler can be customized by
# explicitly constructing the objects and pass to the trainer.
#
# optimizer = Optimizer(torch.optim.Adam(seq2seq.parameters()), max_grad_norm=5)
# scheduler = StepLR(optimizer.optimizer, 1)
# optimizer.set_scheduler(scheduler)

# train
t = SupervisedTrainer(loss=loss, batch_size=32,
                      checkpoint_every=50,
                      print_every=10, expt_dir=opt.expt_dir)

seq2seq = t.train(seq2seq, train,
                  num_epochs=6, dev_data=dev,
                  optimizer=optimizer,
                  teacher_forcing_ratio=0.5,
                  resume=opt.resume)

predictor = Predictor(seq2seq, input_vocab, output_vocab)

In [5]:




# Sample usage:
#     # training
#     python examples/sample.py --train_path $TRAIN_PATH --dev_path $DEV_PATH --expt_dir $EXPT_PATH
#     # resuming from the latest checkpoint of the experiment
#      python examples/sample.py --train_path $TRAIN_PATH --dev_path $DEV_PATH --expt_dir $EXPT_PATH --resume
#      # resuming from a specific checkpoint
#      python examples/sample.py --train_path $TRAIN_PATH --dev_path $DEV_PATH --expt_dir $EXPT_PATH --load_checkpoint $CHECKPOINT_DIR

parser = argparse.ArgumentParser()
parser.add_argument('--train_path', action='store', dest='train_path',
                    help='Path to train data')
parser.add_argument('--dev_path', action='store', dest='dev_path',
                    help='Path to dev data')
parser.add_argument('--expt_dir', action='store', dest='expt_dir', default='./experiment',
                    help='Path to experiment directory. If load_checkpoint is True, then path to checkpoint directory has to be provided')
parser.add_argument('--load_checkpoint', action='store', dest='load_checkpoint',
                    help='The name of the checkpoint to load, usually an encoded time string')
parser.add_argument('--resume', action='store_true', dest='resume',
                    default=False,
                    help='Indicates if training has to be resumed from the latest checkpoint')
parser.add_argument('--log-level', dest='log_level',
                    default='info',
                    help='Logging level.')

opt = parser.parse_args()

LOG_FORMAT = '%(asctime)s %(name)-12s %(levelname)-8s %(message)s'
logging.basicConfig(format=LOG_FORMAT, level=getattr(logging, opt.log_level.upper()))
logging.info(opt)

if opt.load_checkpoint is not None:
    logging.info("loading checkpoint from {}".format(os.path.join(opt.expt_dir, Checkpoint.CHECKPOINT_DIR_NAME, opt.load_checkpoint)))
    checkpoint_path = os.path.join(opt.expt_dir, Checkpoint.CHECKPOINT_DIR_NAME, opt.load_checkpoint)
    checkpoint = Checkpoint.load(checkpoint_path)
    seq2seq = checkpoint.model
    input_vocab = checkpoint.input_vocab
    output_vocab = checkpoint.output_vocab
else:
    # Prepare dataset
    src = SourceField()
    tgt = TargetField()
    max_len = 50
    def len_filter(example):
        return len(example.src) <= max_len and len(example.tgt) <= max_len
    train = torchtext.data.TabularDataset(
        path=opt.train_path, format='tsv',
        fields=[('src', src), ('tgt', tgt)],
        filter_pred=len_filter
    )
    dev = torchtext.data.TabularDataset(
        path=opt.dev_path, format='tsv',
        fields=[('src', src), ('tgt', tgt)],
        filter_pred=len_filter
    )
    src.build_vocab(train, max_size=50000)
    tgt.build_vocab(train, max_size=50000)
    input_vocab = src.vocab
    output_vocab = tgt.vocab

    # NOTE: If the source field name and the target field name
    # are different from 'src' and 'tgt' respectively, they have
    # to be set explicitly before any training or inference
    # seq2seq.src_field_name = 'src'
    # seq2seq.tgt_field_name = 'tgt'

    # Prepare loss
    weight = torch.ones(len(tgt.vocab))
    pad = tgt.vocab.stoi[tgt.pad_token]
    loss = Perplexity(weight, pad)
    if torch.cuda.is_available():
        loss.cuda()

    seq2seq = None
    optimizer = None
    if not opt.resume:
        # Initialize model
        hidden_size=128
        bidirectional = True
        encoder = EncoderRNN(len(src.vocab), max_len, hidden_size,
                             bidirectional=bidirectional, variable_lengths=True)
        decoder = DecoderRNN(len(tgt.vocab), max_len, hidden_size * 2 if bidirectional else hidden_size,
                             dropout_p=0.2, use_attention=True, bidirectional=bidirectional,
                             eos_id=tgt.eos_id, sos_id=tgt.sos_id)
        seq2seq = Seq2seq(encoder, decoder)
        if torch.cuda.is_available():
            seq2seq.cuda()

        for param in seq2seq.parameters():
            param.data.uniform_(-0.08, 0.08)

        # Optimizer and learning rate scheduler can be customized by
        # explicitly constructing the objects and pass to the trainer.
        #
        # optimizer = Optimizer(torch.optim.Adam(seq2seq.parameters()), max_grad_norm=5)
        # scheduler = StepLR(optimizer.optimizer, 1)
        # optimizer.set_scheduler(scheduler)

    # train
    t = SupervisedTrainer(loss=loss, batch_size=32,
                          checkpoint_every=50,
                          print_every=10, expt_dir=opt.expt_dir)

    seq2seq = t.train(seq2seq, train,
                      num_epochs=6, dev_data=dev,
                      optimizer=optimizer,
                      teacher_forcing_ratio=0.5,
                      resume=opt.resume)

predictor = Predictor(seq2seq, input_vocab, output_vocab)

while True:
    seq_str = raw_input("Type in a source sequence:")
    seq = seq_str.strip().split()
    print(predictor.predict(seq))

Using TensorFlow backend.


TypeError: while_loop() got an unexpected keyword argument 'maximum_iterations'